In [1]:
import json
import pandas as pd
import cmath
import xml.etree.ElementTree as et
import numpy as np
from transformations import *


In [2]:
pd.set_option('display.max_columns', 5000)
#pd.set_option('display.max_rows', None)

In [3]:
def getformations(dataframe): 
    df=dataframe['action_name']
    dft=dataframe['team_name']
    list=[]
    team=[]
    for i in range(4):
        x=df.iloc[i]
        teams = dft.iloc[i]
        if ('2'<x[0]<'6'):
            list+=[x]
            team+=[teams]

    team1 = [team[0], list[0]]
    team2 = [team[1], list[1]]
            
    return team1, team2

In [4]:
#instatiate
template_path='template.csv'
df=pd.read_csv(template_path)
df

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type.primary,type.secondary,location.x,location.y,team.id,team.name,team.formation,opponentTeam.id,opponentTeam.name,opponentTeam.formation,player.id,player.name,player.position,pass.accurate,pass.angle,pass.height,pass.length,pass.recipient.id,pass.recipient.name,pass.recipient.position,pass.endLocation.x,pass.endLocation.y,possession.id,possession.duration,possession.types,possession.eventsNumber,possession.eventIndex,possession.startLocation.x,possession.startLocation.y,possession.endLocation.x,possession.endLocation.y,possession.team.id,possession.team.name,possession.team.formation,possession.attack,pass,groundDuel.opponent.id,groundDuel.opponent.name,groundDuel.opponent.position,groundDuel.duelType,groundDuel.keptPossession,groundDuel.progressedWithBall,groundDuel.stoppedProgress,groundDuel.recoveredPossession,groundDuel.takeOn,groundDuel.side,groundDuel.relatedDuelId,infraction.yellowCard,infraction.redCard,infraction.type,infraction.opponent.id,infraction.opponent.name,infraction.opponent.position,possession.attack.withShot,possession.attack.withShotOnGoal,possession.attack.withGoal,possession.attack.flank,possession.attack.xg,carry.progression,carry.endLocation.x,carry.endLocation.y,shot.bodyPart,shot.isGoal,shot.onTarget,shot.goalZone,shot.xg,shot.postShotXg,shot.goalkeeperActionId,shot.goalkeeper.id,shot.goalkeeper.name,possession,aerialDuel.opponent.id,aerialDuel.opponent.name,aerialDuel.opponent.position,aerialDuel.opponent.height,aerialDuel.firstTouch,aerialDuel.height,aerialDuel.relatedDuelId,shot.goalkeeper,infraction.opponent,location


In [5]:
#set all unknown attributes
id=matchId=videoTimestamp=relatedEventId=teamId=opponentTeamId=playerId=passrecipientId=possessionId=possessioneventsNumber = 0
possessioneventIndex=possessionteamId=groundDuelopponentId=groundDuelrelatedDuelId=infractionopponentId=shotgoalkeeperActionId=shotgoalkeeperId=aerialDuelopponentId=aerialDuelrelatedDuelId = 0
passheight =groundduel_take_on= ground_side=None
postshotxg=shotxg=shotgoalzone=possessionattackxg=aerialduelheight=aerialFirsttouch=aerialdueloppheight=np.nan


In [6]:
#carry to do
carry_x=carry_y=carry_prog=np.nan

In [7]:
#load instat file
instat_path = "data/InStat_AXA_Super_2023-06-02_Switzerland. AXA Women's Super League_Servette Chenois_vs_Zurich_2582307_2023-06-02_Servette Chenois_vs_Zurich_2582307_rawdata - Kopie.xml"
instatdf=pd.read_xml(instat_path)
instatdf=instatdf.iloc[2:,:]
instatdf.drop(columns=['column','number','dl', 'id', 'uid', 'action_id', 'player_id', 'team_id', 'standart_id', 'ts', 'position_id', 'opponent_id', 'opponent_team_id','opponent_position_id', 'zone_id', 'zone_dest_id', 'possession_id', 'possession_team_id', 'possession_team_id', 'attack_status_id', 'attack_type_id', 'attack_team_id', 'body_id' ], inplace=True)
#get the goalkeepers
keeperA,keeperB= get_keepers(instatdf)



In [9]:
keeperB

['Seraina Friedli', 'Zurich']

In [ ]:
mask = (instatdf['second']==0.00) & (~instatdf['player_name'].isna()) & (instatdf['action_name']!='Attacking pass accurate')
instatdf = instatdf[~mask]
todrop = ['line-up', 'Substitute player', '1st half','2nd half', 'Players, that created offside trap']
mask = ~instatdf['action_name'].isin(todrop)
instatdf = instatdf[mask]
teamA,teamB=getformations(instatdf)

In [7]:
teamA, teamB

(['Servette Chenois', '4-2-3-1'], ['Zurich', '4-4-2 classic'])

True

In [10]:
instatdf

,action_name,player_name,team_name,half,second,standart_name,position_name,opponent_name,opponent_team_name,opponent_position_name,zone_name,zone_dest_name,pos_x005F_x,pos_y,pos_dest_x005F_x,pos_dest_y,len,possession_name,possession_team_name,possession_time,possession_number,direction,attack_status_name,attack_type_name,attack_flang_id,attack_flang_name,attack_team_name,attack_number,gate_x005F_x,gate_y,body_name,shot_type,touches,shot_handling,assistant_id,assistant_name
50,Attacking pass accurate,Paula Serrano Castano,Servette Chenois,1.0,0.00,Indirect free kick,Attacking midfielder - Central,Nahla Elodie Nakkach,Servette Chenois,Defensive midfielder - Right central,2C,2C,52.5,34.1,39.9,34.1,12.6,Start,Servette Chenois,0.0,1.0,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
153,Ball receiving,Nahla Elodie Nakkach,Servette Chenois,1.0,1.41,Open play,Defensive midfielder - Right central,None,None,None,2C,None,39.9,34.1,NaN,NaN,NaN,Ball possession,Servette Chenois,0.0,1.0,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
154,Attacking pass inaccurate,Nahla Elodie Nakkach,Servette Chenois,1.0,2.51,Open play,Defensive midfielder - Right central,None,None,None,2C,4L,41.8,33.3,105.0,56.3,67.3,Ball possession,Servette Chenois,0.0,1.0,0.9,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
155,Lost balls,Nahla Elodie Nakkach,Servette Chenois,1.0,2.51,Open play,None,None,None,None,4L,None,105.0,56.3,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
156,Ball out of the field,None,Servette Chenois,1.0,8.49,Open play,None,None,None,None,4L,None,105.0,57.8,NaN,NaN,NaN,End,Servette Chenois,8.0,1.0,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634,Attacking pass accurate,Nahla Elodie Nakkach,Servette Chenois,2.0,2927.92,Open play,Defender - Left central,Maeva Jocelyne Clemaron,Servette Chenois,Defensive midfielder - Right central,2L,2L,28.7,54.1,34.8,56.3,6.5,Ball possession,Servette Chenois,0.0,239.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,168.0,NaN,NaN,None,None,None,None,NaN,None
2635,Attacking pass accurate,Maeva Jocelyne Clemaron,Servette Chenois,2.0,2929.40,Open play,Defensive midfielder - Right central,Nahla Elodie Nakkach,Servette Chenois,Defender - Left central,2L,2L,34.8,56.3,32.5,63.2,7.3,Ball possession,Servette Chenois,0.0,239.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,168.0,NaN,NaN,None,None,None,None,NaN,None
2636,Challenge,Nahla Elodie Nakkach,Servette Chenois,2.0,2930.33,Open play,Defender - Left central,Alayah Pilgrim,Zurich,Midfielder - Right,2L,None,32.5,63.2,NaN,NaN,NaN,Ball possession,Servette Chenois,0.0,239.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,168.0,NaN,NaN,None,None,None,None,NaN,None
2637,Ball out of the field,None,Servette Chenois,2.0,2932.14,Open play,None,None,None,None,2L,None,31.2,68.0,NaN,NaN,NaN,End,Servette Chenois,10.0,239.0,NaN,End,Positional attack,1.0,left flank,Servette Chenois,168.0,NaN,NaN,None,None,None,None,NaN,None


In [9]:
mask = (instatdf['second']==0)&(instatdf['action_name']!='Attacking pass accurate')
instatdf = instatdf[~mask] 
#ready for wyscout transformation

In [ ]:
#set the first possession length

In [ ]:
#get the index for the new event
def next_event(df, index): 
    ind = index
    return ind

In [12]:
position_transform('Defender - Right central', teamA[1])

'RCB3'

['5', '7']

In [26]:
"hello"=="hello"

True

In [8]:
current_possession,poss_types, withshot,withshotongoal, withgoal, flank=setnewpossession(instatdf,0) #start x, y, end x, y, possession_length, possessionstart index|possession types, attack variables


In [20]:
None==False

False

In [16]:
pd.isna("44")

False

In [12]:
def create_event(instat, ind, wyscout):
    index_instat, typeprimary, typesecondary = get_event_type(instat, ind)
    action = instat['action_name'].iloc[ind]
    #setup every attribute
    minute, second, matchTimestamp = get_time(instat, ind)
    locx, locy = get_location(instat, ind)
    teamname = instat['team_name'].iloc[ind]
    teamformation = teamA[1] if (teamname==teamA[0]) else teamB[1]
    oppteamname = teamB[0] if (teamname==teamA[0]) else teamA[0]
    oppteamformation = teamB[1] if (teamname==teamA[0]) else teamA[1]
    playername = instat['player_name'].iloc[ind]  #have to change to player names
    playerposition = position_transform(instat['position_name'].iloc[ind], teamformation)
    passlength = instat['len'].iloc[ind]
    possessionlength = instat['possession_time'].iloc[ind]
    possteamname = instat['possession_team_name'].iloc[ind]
    possteamformation = teamA[1] if (possteamname==teamA[0]) else teamB[1]
    possession = instat['possession_name'].iloc[ind]
    keptPoss=None if ('defensive_duel' in typesecondary) else (False if possession=='Transition of possession' or possession=='End' else True)
    stopped_prog=None if ('offensive_duel' in typesecondary) else (True if possession=='Transition of possession' or possession=='End' else False)


    #passing
    if not (pd.isna(instat["pos_dest_x005F_x"].iloc[ind])):
        passaccurate=isaccurate_pass(action)
        passangle=calculate_angle(instat["pos_x005F_x"].iloc[ind], instat["pos_y"].iloc[ind], instat["pos_dest_x005F_x"].iloc[ind], instat["pos_dest_y"].iloc[ind])
        passend_x, passend_y=get_dest_location(instat,ind)
        passrec, passrec_pos_instat = get_pass_recipient(instat, ind)
        passrec_pos = position_transform(passrec_pos_instat, teamformation)
    else:
        passaccurate=passangle=passend_x=passend_y=passrec=passrec_pos=np.nan
    
    #shot
    if (isshot(action)):
        goalkeeper = keeperA[0] if (teamname!=keeperA[1]) else keeperB[0]
        bodypart=bodypart_transform(instat['body_name'].iloc[ind])
        isgoal= True if 'goal' in typesecondary else False
        isontarget= True if (action=='Shot on target' or action == 'Goal') else False
    else:
        bodypart=isgoal=isontarget=np.nan

    #infraction
    if(action=='infraction'):
        isyellow=True if ('yellow_card' in typesecondary) else False
        isred = True if ('red_card' in typesecondary) else False
        infraction_type = 'regular_foul' if action!='Deferred foul' else 'late_card_foul' #there are more types of fouls
        infractionopp = instat['opponent_name'].iloc[ind]
        infractionopp_pos = position_transform(instat['opponent_position_name'].iloc[ind], oppteamformation)
    else:
        isyellow=isred=infraction_type=infractionopp=infractionopp_pos=np.nan

    #duel
    if(typeprimary=='duel'):
        aerialopp=aerialopp_pos=groundopp=groundopp_pos =grounddueltype=np.nan 
        ground_stopped_prog=ground_recov_poss =ground_side=groundkeptposs=groundduel_progressed_with_ball=np.nan

        duelopp = instat['opponent_name'].iloc[ind]
        duelopp_pos = position_transform(instat['opponent_position_name'].iloc[ind], oppteamformation)


        #adjust loss tag

        if ('loss' in typesecondary):
            if(stopped_prog==True):
                typesecondary.remove('loss')



        if ('aerial_duel' in typesecondary):
            aerialopp=duelopp
            aerialopp_pos = duelopp_pos
            
        else:
            
            groundopp=duelopp
            groundopp_pos = duelopp_pos
            grounddueltype = 'dribble' if 'dribble' in typesecondary else ('defensive_duel' if 'defensive_duel' in typesecondary else 'offensive_duel')
            groundkeptposs = keptPoss
            groundduel_progressed_with_ball=groundkeptposs #not so clear what progressed with ball means exactly
            ground_stopped_prog=stopped_prog
            ground_recov_poss =ground_stopped_prog #not so clear
            

    else:
        aerialopp=aerialopp_pos=groundopp=groundopp_pos =grounddueltype=np.nan 
        ground_stopped_prog=ground_recov_poss =ground_side=groundkeptposs=groundduel_progressed_with_ball=np.nan
    #possession
    if (current_possession[5]>ind):
        poss_x=poss_y=endposs_x=endposs_y=np.nan
    else:
        poss_x=current_possession[0]
        poss_y=current_possession[1]
        endposs_x=current_possession[2]
        endposs_y=current_possession[3]
    
    if (possessionlength>0):
        current_possession,poss_types, withshot,withshotongoal, withgoal, flank = setnewpossession(instat, ind+1) 

    
    
    
    new_event = {
        "id": id,
        "matchId": matchId,
        "matchPeriod":get_period(instat,ind),
        "minute":minute,
        "second":second,
        "matchTimestamp":matchTimestamp,
        "videoTimestamp":videoTimestamp,
        "relatedEventId":relatedEventId,
        "shot":np.nan,
        "groundDuel":np.nan,
        "aerialDuel":np.nan,
        "infraction":np.nan,
        "carry":np.nan,
        "type.primary":typeprimary,	
        "type.secondary":typesecondary,
        "location.x":locx,	
        "location.y":locy,	
        "team.id":teamId,	
        "team.name": teamname,	
        "team.formation":teamformation,	
        "opponentTeam.id":opponentTeamId,
        "opponentTeam.name":oppteamname,	
        "opponentTeam.formation":oppteamformation,	
        "player.id":playerId,	
        "player.name":playername,	
        "player.position":playerposition,	
        "pass.accurate":passaccurate,	
        "pass.angle":passangle,	
        "pass.height":passheight,	
        "pass.length":passlength,	
        "pass.recipient.id":passrecipientId,	
        "pass.recipient.name":passrec,	
        "pass.recipient.position":passrec_pos,
        "pass.endLocation.x":passend_x,	
        "pass.endLocation.y":passend_y,	
        "possession.id":possessionId,	
        "possession.duration":current_possession[4],	
        "possession.types":poss_types,	
        "possession.eventsNumber":possessioneventsNumber,	
        "possession.eventIndex":possessioneventIndex,	
        "possession.startLocation.x":poss_x,
        "possession.startLocation.y":poss_y,	
        "possession.endLocation.x":endposs_x,	
        "possession.endLocation.y":endposs_y,	
        "possession.team.id":possessionteamId,	
        "possession.team.name":possteamname,	
        "possession.team.formation":possteamformation,	
        "possession.attack":np.nan,	
        "pass":np.nan,
        "groundDuel.opponent.id":groundDuelopponentId,	
        "groundDuel.opponent.name":groundopp,	
        "groundDuel.opponent.position":groundopp_pos,	
        "groundDuel.duelType":grounddueltype,	
        "groundDuel.keptPossession":groundkeptposs,	
        "groundDuel.progressedWithBall":groundduel_progressed_with_ball,	
        "groundDuel.stoppedProgress":ground_stopped_prog,	
        "groundDuel.recoveredPossession":ground_recov_poss,	
        "groundDuel.takeOn":groundduel_take_on,	
        "groundDuel.side":ground_side,
        "groundDuel.relatedDuelId":groundDuelrelatedDuelId,	
        "infraction.yellowCard":isyellow,	
        "infraction.redCard":isred,	
        "infraction.type":infraction_type,	
        "infraction.opponent.id":infractionopponentId,	
        "infraction.opponent.name":infractionopp,	
        "infraction.opponent.position":infractionopp_pos,	
        "possession.attack.withShot":withshot,	
        "possession.attack.withShotOnGoal":withshotongoal,	
        "possession.attack.withGoal":withgoal,	
        "possession.attack.flank":flank,	
        "possession.attack.xg":possessionattackxg,	
        "carry.progression":carry_prog,	
        "carry.endLocation.x":carry_x,	
        "carry.endLocation.y":carry_y,	
        "shot.bodyPart":bodypart,	
        "shot.isGoal":isgoal,	
        "shot.onTarget":isontarget,	
        "shot.goalZone":shotgoalzone,	
        "shot.xg": shotxg,	
        "shot.postShotXg":postshotxg,	
        "shot.goalkeeperActionId":shotgoalkeeperActionId,	
        "shot.goalkeeper.id":shotgoalkeeperId,	
        "shot.goalkeeper.name":goalkeeper,	
        "possession":np.nan,	
        "aerialDuel.opponent.id":aerialDuelopponentId,	
        "aerialDuel.opponent.name":aerialopp,	
        "aerialDuel.opponent.position":aerialopp_pos,	
        "aerialDuel.opponent.height":aerialdueloppheight,	
        "aerialDuel.firstTouch":aerialFirsttouch,	
        "aerialDuel.height":aerialduelheight,	
        "aerialDuel.relatedDuelId":aerialDuelrelatedDuelId,	
        "shot.goalkeeper":np.nan,	
        "infraction.opponent":np.nan,	
        "location":np.nan
    }



    wdf = pd.concat([wyscout, pd.DataFrame([new_event])], ignore_index=True)
    #check when second event has to be generated
    if (typeprimary=='duel'):
        new_event_2=create_second_duel_event(new_event, keptPoss, stopped_prog)
        wdf = pd.concat([wdf, pd.DataFrame([new_event])], ignore_index=True)
    elif (action=='Shot on target' or action=='Goal'):
        new_event_2=create_second_shot_event(new_event)
        wdf = pd.concat([wdf, pd.DataFrame([new_event])], ignore_index=True)
    
    
    return index_instat, wdf

In [21]:
l=[1,2,3]
l.remove(1)
l

[2, 3]

In [ ]:
matchend=True
index = 0   
while (matchend):
    index, df = create_event(index, df)
    if(instatdf['action_name'].iloc[index]=='Match end'):
        break


In [ ]:
def parseaction(instat, ind, wyscout):
    newind=ind
    starttime
    return newind

In [ ]:
instatdf

In [12]:
some_row={'A':2, 'B':4, 'C':44}
some_row['B']=2323


In [13]:
some_row

{'A': 2, 'B': 2323, 'C': 44}